In [37]:
import pandas as pd
date = 20211205
category = '사회'
df = pd.read_json(f'./articles/daum_articles_{date}_{category}.json')
df.drop(['id', 'category', 'publish_date', 'extractive', 'abstractive'], axis=1, inplace=True)
result = ''
df['result'] = ''
for i in range(len(df['article'])):
    result = []
    for j in range(len(df['article'][i])):
        result+=(df['article'][i][j]['sentence']).strip()
    df['article'][i] =''.join(result)
    df['result'][i] = df['title'][i]+' '+df['article'][i]

In [76]:
# 23초

from konlpy.tag import Okt
okt = Okt()
df['result_nouns'] = ''
# df['title_nouns'] = ''

for i in range(len(df['result'])):
    tmp = okt.nouns(df['result'][i])
    df['result_nouns'][i] = tmp

# for i in range(len(df['title'])):
#     tmp = okt.nouns(df['title'][i])
#     df['title_nouns'][i] = tmp



<ipython-input-76-c1d347ebbcdc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result_nouns'][i] = tmp


In [77]:
df.to_pickle(f"daum_{category}_news.pkl")

In [78]:
import string
exclude = string.punctuation + '‘’·“”…'

def preprocess_title(sent):
    return "".join(chr for chr in sent if chr not in exclude)

titles = list(map(preprocess_title, df.iloc[:,0]))
print(exclude)
print(titles[:3])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~‘’·“”…
['김상호 하남시장 생활속 에너지절약 챌린지 동참', '창원시 별빛 쏟아지는 경화역 철길 따라 걸어요', '2년만에 불밝힌 남강유등 진주남강유등코리아드라마축제 개막']


In [79]:
drop_index_list = [] # 지워버릴 index를 담는 리스트
for i, row in df.iterrows():
    temp_nouns = row['result_nouns']
    if len(temp_nouns) == 0: # 만약 명사리스트가 비어 있다면
        drop_index_list.append(i) # 지울 index 추가
        
df = df.drop(drop_index_list) # 해당 index를 지우기

# index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
df.index = range(len(df))

len(drop_index_list)

0

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['result_nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
# tfidf_vectorizer.fit(text)
# vector = tfidf_vectorizer.transform(text).toarray()
vector =  tfidf_vectorizer.fit_transform(text).toarray()
vector

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [87]:
from sklearn.cluster import DBSCAN
import numpy as np

vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
model = DBSCAN(eps=0.3 ,min_samples=5, metric = "cosine")
result = model.fit_predict(vector)
df['cluster_label'] = result
 
df.head()

,title,article,result,result_nouns,cluster_label
0,김상호 하남시장 '생활속 에너지절약' 챌린지 동참,이번 챌린지 캠페인은 9월7일 양승조 충남도지사가 시작했다. △전자타이머 콘센트 활...,김상호 하남시장 '생활속 에너지절약' 챌린지 동참 이번 챌린지 캠페인은 9월7일 양...,"[김상호, 하남시, 생활, 속, 에너지, 절약, 린지, 동참, 이번, 린지, 캠페인...",-1
1,"창원시 ""별빛 쏟아지는 경화역 철길 따라 걸어요""",벚꽃 명소로 알려진 창원 진해 경화역이 별빛 경관조명으로 빛난다.창원시는 지난 11...,"창원시 ""별빛 쏟아지는 경화역 철길 따라 걸어요"" 벚꽃 명소로 알려진 창원 진해 경...","[창원시, 별빛, 경화역, 철길, 벚꽃, 명소, 창원, 진해, 경화역, 별빛, 관조...",-1
2,"""2년만에 불밝힌 남강유등"" 진주남강유등·코리아드라마축제 개막",대한민국 대표 문화관광축제인 진주남강유등축제와 대한민국 대표 드라마 축제인 코리아드...,"""2년만에 불밝힌 남강유등"" 진주남강유등·코리아드라마축제 개막 대한민국 대표 문화관...","[불, 남, 강유, 등, 진주, 남, 강유, 등, 코리아, 드라마, 축제, 개막, ...",-1
3,김상호 하남시장 '세이브 아프간 위민' 챌린지 동참,"김상호 시장은 ‘인권보호! 국가도, 인종도, 성별도 예외는 없습니다. 아프간 여성의...",김상호 하남시장 '세이브 아프간 위민' 챌린지 동참 김상호 시장은 ‘인권보호! 국가...,"[김상호, 하남시, 세이브, 아프간, 위민, 린지, 동참, 김상호, 시장, 인권, ...",-1
4,"화이자 2차 접종 닷새만에 쓰러져 '급성 백혈병' 진단 초6..母 ""RH+ A형 혈...",5일 연합뉴스에 따르면 경북 칠곡군에 사는 초등 6년 A양의 부모는 지난달 24일 ...,"화이자 2차 접종 닷새만에 쓰러져 '급성 백혈병' 진단 초6..母 ""RH+ A형 혈...","[화이자, 차, 접종, 닷새, 급성, 백혈병, 진단, 초, 형, 혈소판, 급구, 호...",-1


In [88]:
# predict = pd.DataFrame(model.fit_predict(feature))
# predict.columns=['predict']

# # concatenate labels to df as a new column
# r = pd.concat([feature,predict],axis=1)

In [89]:
km_cluster = model # 거리 계산 식으로는 Cosine distance를 이용
cluster_label = km_cluster.labels_
# cluster_centers = km_cluster.cluster_centers_

km_cluster


DBSCAN(eps=0.3, metric='cosine')

In [90]:
# import mglearn

# mglearn.plots.plot_dbscan()

In [91]:
for cluster_num in set(result):
    # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
    if(cluster_num == -1 or cluster_num == 0): 
        continue
    else:
        print("cluster num : {}".format(cluster_num))
        temp_df = df[df['cluster_label'] == cluster_num] # cluster num 별로 조회
        for title in temp_df['title']:
            print(title) # 제목으로 살펴보자
        print()
print("불가능 / 총개수 : ", len(df[df['cluster_label'] == -1]) + len(df[df['cluster_label'] == 0]),'/', len(df))      

cluster num : 1
25년 뒤, 국내 대학 절반이 사라진다
"25년 내 국내 대학 절반 사라질 수도"
"2046년 국내 대학 절반 이상 사라진다"..현재 385개에서 190개만 생존
학령인구 감소로 생존 기로 놓인 지방대.."25년 뒤 절반 이상 사라진다"
"25년 뒤 국내 대학 절반 사라질 수도"..미래전망 전문가 포럼
"대책 절실하다"..25년 내 대학 절반 이상 소멸

cluster num : 2
이성윤 수사팀 "대검 감찰부터 압수수색해야" 공수처에 의견서
"총장님, '이성윤 공소장 유출' 진상조사 발표 지시하십시오"
수원지검 수사팀 "이성윤 공소장 유출한 적 없어" 공수처에 의견서 제출
이성윤 수사팀 "대검 감찰부터 압수수색하라" 공수처에 의견서
'이성윤 공소장유출' 공수처 수사에..수원지검팀 "김오수 총장, 도와달라"
"공수처 표적수사" 수원지검 수사팀 '발끈'.."김오수 입장내달라" 강수
"공수처의 수사는 부당" 전 수원지검 수사팀 의견서 제출
이성윤 수사팀 "대검 감찰부 자료공개해야..김오수 총장 나서달라"
[단독] "대검을 압수수색하라" 이성윤 수사팀, 공수처에 의견서

cluster num : 3
'격전지' 철원 화살머리고지에 평화기념관
한국전쟁 격전지 철원 화살머리고지에 '평화기념관' 건립된다
철원 화살머리고지에 평화기념관 건립..2025년까지 283억 투입
6·25격전지 철원 화살머리고지에 평화기념관 들어선다
철원 화살머리고지에 평화의 공간 들어선다

cluster num : 4
"택시기사님을 모십니다" 첫 대규모 취업박람회
법인택시 운수종사자 취업박람회 열린다
심야 승차난에..택시기사 취업박람회까지 여는 서울시
"심야택시난 해소" 서울 법인택시 첫 대규모 취업박람회
[서울] 서울시-택시업계, 인력확보 위해 법인택시 취업박람회 개최
서울시-택시업계, 인력확보 나선다..법인택시 취업박람회
서울시-택시업계, 운수종사자 대대적 확충 나선다..첫 '취업박람회'
"법인택시 종사자 찾습니다"..서울 첫 법인택시 취업박람회 8일 개최
서울시-택

# keyword 뽑기 추가

In [92]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape : ', cluster_centers.shape)
print(cluster_centers)

AttributeError: 'DBSCAN' object has no attribute 'cluster_centers_'

In [ ]:
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 제목 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['title']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['title'] = filenames
        
    return cluster_details

In [ ]:
def print_cluster_details(cluster_details):    
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Title :',cluster_detail['title'][:7])
        print('==================================================')

In [ ]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=topic_df,\
                                  feature_names=feature_names, clusters_num=clusters_num, top_n_features=10 )
print_cluster_details(cluster_details)

In [ ]:
WordVoca_list = []
for item in TitDesc_okt:
    WordVoca_list.append(item.split(' '))

# split된 2차원 리스트 중에서 한 글자짜리 단어들을 모두 제외시키고 WordVoca 리스트 생성
# Word2Vec 모델 학습 데이터로 활용됨
WordVoca = []
for i in range(len(WordVoca_list)):
    element = []
    for j in range(len(WordVoca_list[i])):
        if len(WordVoca_list[i][j]) > 1:
            element.append(WordVoca_list[i][j])
    WordVoca.append(element)

In [ ]:
from gensim.models import word2vec

def WordSimilarity(word, count):
    model = Word2Vec(sentences=WordVoca, size=100, window=5, min_count=5, workers=4, sg=1)
    model_result = model.most_similar(positive=[word], topn=count)

    Similarity_df = pd.DataFrame(model_result, columns=[word,'Similarity'])
    print('{0}과 유사한 단어 Top {1} :'.format(word, count))
    print(Similarity_df)

In [ ]:
# WordSimilarity('더존', 10)

In [ ]:
df_sorted_by_values = topic_df.sort_values(by='cluster_label' ,ascending=True)
df_sorted_by_values.to_csv("daum_1205_문화_clustering.csv", index=False)

df_sorted_by_values

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
cnt_vect = CountVectorizer(tokenizer=LemNormalize,)

In [ ]:
dtm = tfidf_vect.fit_transform(topic_df['TokenizedTitDesc'])

In [ ]:
n_topics = 10

lda = LatentDirichletAllocation(n_components=n_topics)
lda.fit(dtm)

In [ ]:
names = tfidf_vect.get_feature_names()
topics = dict()

for idx, topic in enumerate(lda.components_):
    vocab = []
    for i in topic.argsort()[:-(30-1):-1]:
        vocab.append((names[i], topic[i].round(2)))
    print('주제 : {0}'.format(idx+1))
    print([(names[i], topic[i].round(2)) for i in topic.argsort()[:-(30-1):-1]])

In [ ]:
feature_vect_cnt = cnt_vect.fit_transform(topic_df['TokenizedTitDesc'])
lda2 = LatentDirichletAllocation(n_components=n_topics, topic_word_prior=0.01, doc_topic_prior=0.001)
lda2.fit(dtm)

names = cnt_vect.get_feature_names()
topics_word = dict()
n_words = 10

for idx, topic in enumerate(lda2.components_):
    vocab = []
    for i in topic.argsort()[:-(30-1):-1]:
        vocab.append((names[i], topic[i].round(2)))
    topics_word[idx+1] = [(names[i], topic[i].round(2)) for i in topic.argsort()[:-(n_words-1):-1]]
max_dict = dict()

for idx, vec in enumerate(lda2.transform(dtm)):
    t = vec.argmax()
    if (t not in max_dict):
        max_dict[t] = (vec[t], idx)
    else:
        if (max_dict[t][0] < vec[t]):
            max_dict[t] = (vec[t], idx)
sorted_news = sorted(max_dict.items(), key = lambda x: x[0], reverse=False)

for key, value in sorted_news:
    print('주제 {0}: {1}]'.format(key+1, topics_word[key+1]))
    print('[주제 {}의 대표 기사 : {}]\n{}\n\n'.format(key+1, value[0], topic_df['title'][value[1]]))